In [ ]:
%load_ext tensorboard
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from datetime import datetime

%tensorboard --logdir logs/scalars/



tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir= "logs/scalars/" + datetime.now() .strftime("%Y%m%d-%H%M%S"),
    histogram_freq=0,
    write_graph=True,
    write_grads=True
    )


n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 50    # Size of the train dataset
n_test = 30     # Size of the test dataset

SAVE_PATH = "quanvolution/" # Data saving folder
PREPROCESS = False           # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator



mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()



# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]




In [ ]:

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

n_qubits = 4

dev = qml.device("default.qubit", wires=n_qubits)
# Random circuit parameters
random_weights = np.random.uniform(high=2 * np.pi, size=(n_layers, n_qubits))

@qml.qnode(dev)
def qnode(inputs, weights):
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Random quantum circuit
    RandomLayers(weights, wires=list(range(n_qubits)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]



In [ ]:
class ConvQLayer(qml.qnn.KerasLayer):
    
    def call(self, inputs):
        
        out = tf.Variable(tf.zeros((n_qubits, 14, 14)))

        # Loop over the coordinates of the top-left pixel of 2X2 squares
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                # Process a squared 2x2 region of the image with a quantum circuit
                q_in = tf.stack(
                    [
                        inputs[:,j, k, 0],
                        inputs[:,j, k + 1, 0],
                        inputs[:,j + 1, k, 0],
                        inputs[:,j + 1, k + 1, 0]
                    ]
                )
                q_results = super().call(q_in)
                # Assign expectation values to different channels of the output pixel (j/2, k/2)
                out[:,j // 2, k // 2] = q_results
        return out

weight_shapes = {"weights": (n_layers, n_qubits)}

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=1)

qlayer.set_weights([random_weights]) 

qlayer.trainable = False

In [ ]:
qtens = qlayer(train_images[0])

In [14]:
tf.expand_dims(qtens, axis = 0).shape

TensorShape([1, 28, 28, 4])

In [ ]:
pict_layer = keras.layers.Conv2D(32,(3,3),activation='relu',bat)

In [10]:
pict = pict_layer([[qtens]])

ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (28, 28, 4)